# Get Programming with F# by [Isaac Abraham](https://github.com/isaacabraham)

## “Modeling relationships in F#”

When Isaac Abraham uses the word _relationships_, he is referring to OOP/C# things like:

- _hierarchical_, inheritance relationships
- _lateral_, (interface-based) composition relationships
- database-entity, (mostly) one-to-many relationships ([Entity Framework](https://docs.microsoft.com/en-us/ef/) models)

I have only my ignorance to blame when the author introduces F# _discriminated unions_ [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/discriminated-unions)] as the equivalent/replacement of _hierarchical_, OOP inheritance relationships! Without this introduction, I am sure I would continue to fail to see the Microsoft documentation entry, “[Using Discriminated Unions Instead of Object Hierarchies](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/discriminated-unions#using-discriminated-unions-instead-of-object-hierarchies)” and its `Shape` code sample:

In [ ]:
#!fsharp

type Shape =
| Circle of float
| EquilateralTriangle of double
| Square of double
| Rectangle of double * double

This `Shape` can be thought of (in a C# way) as a “base class” that can _polymorphose_ into one of four ‘things’:

1. `Circle`
2. `EquilateralTriangle`
3. `Square`
4. `Rectangle`

The `of` keyword, listed in this [big-ass table from Microsoft](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/keyword-reference#f-keyword-table), allows us to define fields on these four ‘things’—and the `of` syntax we are seeing above is the _anonymous field_ variety:

>Unnamed fields are referred to as anonymous fields.
>
>—[Microsoft](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/discriminated-unions)

From a C# (or even a Typescript) perspective, discriminated unions is the line in the sand where you decide whether you like F# or not. It _feels_ like you will not really be using F# when you avoid using discriminated unions! I find the terse, expressive power here extremely attractive (but I can also see how hard-core OOP rock stars would walk away from this, determined to _not_ understand what is happening here).

What is happening here, for example, is this anonymous-field `of` syntax allows us to ‘delay’ (at design-time) _naming_ our fields until they are used in a function:

In [ ]:
#!fsharp

let pi = 3.141592654

let area myShape =
    match myShape with
    | Circle radius -> pi * radius * radius // unwrapping the name `radius` from `Circle`
    | EquilateralTriangle s -> (sqrt 3.0) / 4.0 * s * s
    | Square s -> s * s
    | Rectangle (h, w) -> h * w // unwrapping the names `h` and `w` from `Rectangle`

let radius = 15.0
let myCircle = Circle(radius)
printfn "Area of circle that has radius %f: %f" radius (area myCircle)

let squareSide = 10.0
let mySquare = Square(squareSide)
printfn "Area of square that has side %f: %f" squareSide (area mySquare)

let height, width = 5.0, 10.0
let myRectangle = Rectangle(height, width)
printfn "Area of rectangle that has height %f and width %f is %f" height width (area myRectangle)

Area of circle that has radius 15.000000: 706.858347
Area of square that has side 10.000000: 100.000000
Area of rectangle that has height 5.000000 and width 10.000000 is 50.000000


The `area` function is, according to Microsoft documentation at the time of this writing, _unwrapping_ (exposing and naming) anonymous fields from `Shape` in its `match` expression. These names do not exist until they are needed! This is awesome! This is the F# answer to the Typescript answer to the question, Can we define types better in JavaScript?


The following question sounds like someone is [micro-dosing](https://www.forbes.com/sites/amandasiebert/2020/11/13/microdosing-psychedelics-is-trendy-but-does-it-work-heres-what-science-says/?sh=117fdfe6cf7c) a bit too much:

_Did you ever want to define class hierarchies with regular expressions?_

Most hard-core C# developers (many of them have considered themselves my “manager”) would scoff at such a question but be too dismissive to really think about the difference between F# `match` expressions and regular expressions. They would fall back into stating our `Shape` types as:

In [ ]:
public abstract class Shape
{
    protected virtual double GetArea()
    {
        throw new NotImplementedException();
    }
}

public class Circle : Shape
{
    public Circle(double radius) => _radius = radius;

    protected override double GetArea() => Math.PI * _radius * _radius;

    double _radius;
}

public class EquilateralTriangle : Shape
{
    public EquilateralTriangle(double side) => _side = side;

    protected override double GetArea() => Math.Sqrt(3.0)/4.0 * _side * _side;

    double _side;
}

public class Square : Shape
{
    public Square(double side) => _side = side;

    protected override double GetArea() => _side * _side;

    double _side;
}

public class Rectangle : Shape
{
    public Rectangle(double height, double width){
        _height = height;
        _width = width;
    }

    protected override double GetArea() => _height * _width;

    double _height;
    double _width;
}

## the Microsoft docs might call them “anonymous fields” but these are _not_ members of F# records

When we look at `Shape`, we must remind ourselves that it has _no fields_ to share with the members of the discriminated union. And we must also remember that Discriminated unions in F# can be compositions of what we might call _named tuples_ in C# [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/builtin-types/value-tuples#tuple-field-names)]. These named tuples are closer to the `struct` than to the `class` and we must remember that the `struct` does _not_ support inheritance [📖 [docs](https://docs.microsoft.com/en-us/dotnet/csharp/fundamentals/object-oriented/inheritance)].

On page 252 of the book, Isaac Abraham shows how record fields can be shared with a discriminated union by including the DU as a member of the record. Frankly, this was not the class hierarchy I was looking for! But I have myself to blame because `*` denotes tuples, baked in the DU syntax. Apparently, as of this writing, you can only express a DU in terms of tuples—not in terms of records.

_Did you ever want to define class hierarchies without inheritance which means we are talking about a group of tuples that can optionally have field names which cannot be shared?_

Most C# developers (including myself) would soberly answer _no_ to the above question. But after accepting the fact that the DU is more about an advanced application of tuples, we should still find this language feature useful (but not as powerful as we initially thought).

## other names for discriminated unions

Isaac Abraham clarifies:

>The standard functional programming answer to modeling _is-a_ relationships is by using a _discriminated union_ (DU). There are other names for it, such as _sum types_, _case classes_ in Scala, or _algebraic data types_ for people who want to sound smart.


## polymorphism is _not_ confined to OOP

I admit freely that I was one of those C# developers (without a degree in computer science) who just assumed that [polymorphism](https://en.wikipedia.org/wiki/Polymorphism_(computer_science)) is only possible in object-oriented programming (OOP). And, when I use the word _was_, I am talking about the day _before_ I started writing these notes!

According to Wikipedia, the type of polymorphism related to DUs is polytypism, leading to the subject of [_generic programming_](https://en.wikipedia.org/wiki/Generic_programming#Functional_languages).

The type of polymorphism C# developers are used to is subtype polymorphism or inclusion polymorphism—which means the word _subtyping_ is not just c-like programmer slang: it is a reference to knowledge of computer science!


## discriminated unions can make a developer use F# exclusively for Web projects

The [Elmish module](https://elmish.github.io/elmish/) sits on top of the [Fable](https://fable.io/) F#-to-Javascript compiler. Elmish and Fable are part of the [SAFE stack initiative](https://www.compositional-it.com/news-blog/introducing-the-safe-stack/), developed and promoted by Isaac Abraham himself.

The core concept of Elmish is the _message_, “an event representing a change (delta) in the state of your application.” This means an Elmish-based web application will have no interactivity without this message which is represented by—you guessed it—a discriminated union.

Because discriminated unions are so relatively terse and compact with respect to OOP-based languages, whipping up _formal_ types for every single interaction in your application becomes easy, making the application easier to maintain because _all developer intent_ is _denoted_ instead of _connoted_.

## is it _all_ about pattern matching and discriminated unions?

There are 22 lessons left in Isaac Abraham’s book. As of now (lesson 21), I assume that being a “good” F# programmer means you enjoy thinking about and studying:

1. pattern matching
2. discriminated unions

There are tons of other F# features but I currently assume that these two are the basics.

Lesson 21 goes on to explore a few DU patterns:

- nested DUs
- shared fields
- how F# enums look like DUs (see [lesson 8](https://github.com/BryanWilhite/jupyter-central/blob/master/get-programming-with-f-sharp/08-capstone-01.ipynb))

The Microsoft documentation adds [_recursive discriminated unions_](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/discriminated-unions#using-discriminated-unions-for-tree-data-structures) and warns:

>You cannot use a recursive type definition with a multicase `struct` Discriminated Union.

…which implies there is such a thing as a `struct` _discriminated union_ [📖 [docs](https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/discriminated-unions#struct-discriminated-unions)], likely needed to interoperate with other .NET languages.

## the role of discriminated unions in F# design patterns

Because discriminated unions express inheritance it should follow that any OOP design patterns using inheritance should translate quite well to F#. I am certainly not the first to realize this! In 2013, Tao Liu wrote _F# for C# Developers_ [[Microsoft Press](https://www.microsoftpressstore.com/store/f-for-c-sharp-developers-9780735670211)]—and a chapter from the book, 
“[F# and Design Patterns for C# Developers](https://www.microsoftpressstore.com/articles/article.aspx?p=2224057),” addresses this topic.

@[BryanWilhite](https://twitter.com/BryanWilhite)


In [ ]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.255902+b0afbdc47dec91e62b0c5cb587a0f2c24242eca8Build date: 2021-11-09T18:27:50.0000000Zhttps://github.com/dotnet/interactive
